In [4]:
#!pip install xgboost

In [5]:
##from google.colab import drive
##drive.mount('/content/drive')
##with open("/content/drive/MyDrive/Knjiga.tsv", encoding="utf8") as dat:
##    linije_teksta=dat.readlines()
##dat.closed
##dat.closed
##dat.closed

In [98]:
with open("Knjiga.tsv", encoding="utf8") as dat:
    linije_teksta=dat.readlines()
dat.closed
dat.closed
dat.closed

True

In [99]:
## razdvajanje teksta od oznaka
tekst=[]
oznake=[]
for linija in linije_teksta:
    try:
        linija_l=linija.split('\t')
        oznake.append(linija_l[1].rstrip())
        tekst.append(linija_l[0])
    except:
        continue

In [100]:
oznake=[x if x != "p" else "pozitivno" for x in oznake]
print(set(oznake))

{'pozitivno', 'negativno', 'neutralno'}


In [101]:
import pandas as pd

df = pd.DataFrame({"tekst": tekst, "oznake": oznake})
df["oznake"].value_counts()

oznake
neutralno    2709
pozitivno     364
negativno      84
Name: count, dtype: int64

## Treniranje modela

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# kopirano s https://dabar.srce.hr/faq-page/254 bez vrlo
hr_stop_rijeci=['a','ako','ali','bi','bih','bila','bili','bilo','bio',
                'bismo','biste','biti','bumo','da','do','duž','ga',
                'hoće','hoćemo','hoćete','hoćeš','hoću','i','iako',
                'ih','ili','iz','ja','je','jedna','jedne','jedno',
                'jer','jesam','jesi','jesmo','jest','jeste','jesu',
                'jim','joj','još','ju','kada','kako','kao','koja',
                'koje','koji','kojima','koju','kroz','li','me','mene',
                'meni','mi','mimo','moj','moja','moje','mu','na','nad',
                'nakon','nam','nama','nas','naš','naša','naše','našeg',
                'ne','nego','neka','neki','nekog','neku','nema','netko',
                'neće','nećemo','nećete','nećeš','neću','nešto','ni',
                'nije','nikoga','nikoje','nikoju','nisam','nisi',
                'nismo','niste','nisu','njega','njegov','njegova',
                'njegovo','njemu','njezin','njezina','njezino','njih',
                'njihov','njihova','njihovo','njim','njima','njoj','nju',
                'no','o','od','odmah','on','ona','oni','ono','ova','pa',
                'pak','po','pod','pored','prije','s','sa','sam','samo',
                'se','sebe','sebi','si','smo','ste','su','sve','svi',
                'svog','svoj','svoja','svoje','svom','ta','tada','taj',
                'tako','te','tebe','tebi','ti','to','toj','tome','tu',
                'tvoj','tvoja','tvoje','u','uz','vam','vama','vas','vaš',
                'vaša','vaše','već','vi','za','zar','će','ćemo','ćete',
                'ćeš','ću','što']

vectorizer = TfidfVectorizer(encoding="utf8", lowercase=True, analyzer="word",
                             stop_words=hr_stop_rijeci)
tekst=vectorizer.fit_transform(tekst)

skaliranje = StandardScaler(with_mean=False)
tekst=skaliranje.fit_transform(tekst)
le = LabelEncoder()
le.fit(df["oznake"])
print(dict(zip(le.classes_,le.transform(le.classes_))))
oznake = le.transform(df["oznake"])

# Split the data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(
    tekst, oznake, test_size=0.3, random_state=42, stratify=oznake)

# Convert text data into numerical features using TF-IDF
#vectorizer = TfidfVectorizer()
#X_train = vectorizer.fit_transform(X_train)
#X_test = vectorizer.transform(X_test)

{'negativno': 0, 'neutralno': 1, 'pozitivno': 2}


## XGBoost

In [103]:
import xgboost as xgb

# Train the XGBoost model
model = xgb.XGBClassifier(n_jobs=-1)
model.fit(X_train, Y_train)

# Make predictions on the test set
pred = model.predict(X_test)

In [104]:
# Evaluate the model performance
from sklearn.metrics import accuracy_score, f1_score
točnost = accuracy_score(Y_test, pred)
print("Točnost: {:.2f}%".format(točnost * 100))
f1 = f1_score(Y_test, pred, average='macro')
print("F1: {:.2f}%".format(f1 * 100))

Točnost: 85.86%
F1: 41.02%


### Predviđanje

In [105]:
rečenice=["Volim kavu", "Ne volim kavu"]
X_vektori=vectorizer.transform(rečenice)
pred=model.predict(X_vektori)
le.inverse_transform(pred)

array(['neutralno', 'neutralno'], dtype=object)

## Pohrana modela

In [106]:
import pickle
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer

model_file_name = "xgb_reg.pkl"
vectorizer_file_name = 'vectorizer.pk'

# save
pickle.dump(model, open(model_file_name, "wb"))
with open(vectorizer_file_name, 'wb') as fin:
    pickle.dump(vectorizer, fin)

# load
xgb_model_učitano = pickle.load(open(model_file_name, "rb"))
vectorizer_učitano = pickle.load(open(vectorizer_file_name, "rb"))

# predict
pred_učitano = xgb_model_učitano.predict(
    vectorizer_učitano.transform(rečenice))
le.inverse_transform(pred_učitano)

array(['neutralno', 'neutralno'], dtype=object)

## SVM

In [107]:
#sigmoid, C=4 => 86.08%, 33,19%
#linear, C=2 => 85.65%, 30.76%
#poly, C=2 => 85.86%, 30.80%

In [108]:
from sklearn import svm
model=svm.SVC(kernel="sigmoid", C=4)
model.fit(X_train, Y_train)

pred=model.predict(X_test)

In [109]:
from sklearn.metrics import accuracy_score, f1_score
točnost = accuracy_score(Y_test, pred)
print("Točnost: {:.2f}%".format(točnost * 100))
f1 = f1_score(Y_test, pred, average='macro')
print("F1: {:.2f}%".format(f1 * 100))

Točnost: 86.18%
F1: 33.19%


### Predviđanje

In [110]:
rečenice=["Volim kavu", "Ne volim kavu"]
X_vektori=vectorizer.transform(rečenice)
pred=model.predict(X_vektori)
le.inverse_transform(pred)

array(['neutralno', 'neutralno'], dtype=object)

## KNN

In [111]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=4)
model.fit(X_train, Y_train)

pred=model.predict(X_test)

In [112]:
from sklearn.metrics import accuracy_score, f1_score
točnost = accuracy_score(Y_test, pred)
print("Točnost: {:.2f}%".format(točnost * 100))
f1 = f1_score(Y_test, pred, average='macro')
print("F1: {:.2f}%".format(f1 * 100))

Točnost: 85.86%
F1: 30.80%


### Predviđanje

In [113]:
rečenice=["Volim kavu", "Ne volim kavu"]
X_vektori=vectorizer.transform(rečenice)
pred=model.predict(X_vektori)
le.inverse_transform(pred)

array(['neutralno', 'neutralno'], dtype=object)

## Naive Bayes

In [114]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model.fit(X_train.toarray(), Y_train)

pred=model.predict(X_test.toarray())

In [115]:
from sklearn.metrics import accuracy_score, f1_score
točnost = accuracy_score(Y_test, pred)
print("Točnost: {:.2f}%".format(točnost * 100))
f1 = f1_score(Y_test, pred, average='macro')
print("F1: {:.2f}%".format(f1 * 100))

Točnost: 75.84%
F1: 33.89%


## TensorFlow

In [23]:
import tensorflow as tf

model=tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

2024-05-30 11:51:02.381722: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-30 11:51:02.666210: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 11:51:02.666277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 11:51:02.721389: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 11:51:02.840535: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-30 11:51:02.842288: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [24]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=["accuracy"])

In [25]:
model.fit(X_train, Y_train, batch_size=32, epochs=20)

Epoch 1/20


/home/nxlinux/.local/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


70/70 [==============================] - 1s 2ms/step - loss: 0.0000e+00 - accuracy: 0.8013
Epoch 2/20
70/70 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.8592
Epoch 3/20
70/70 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.8601
Epoch 4/20
70/70 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.8615
Epoch 5/20
70/70 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.8637
Epoch 6/20
70/70 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.8660
Epoch 7/20
70/70 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.8710
Epoch 8/20
70/70 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.8710
Epoch 9/20
70/70 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.8728
Epoch 10/20
70/70 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accurac

In [26]:
from sklearn.metrics import accuracy_score, f1_score
točnost = accuracy_score(Y_test, pred)
print("Točnost: {:.2f}%".format(točnost * 100))
f1 = f1_score(Y_test, pred, average='macro')
print("F1: {:.2f}%".format(f1 * 100))

Točnost: 75.84%
F1: 33.89%
